### ConversationBufferMemory

- 대화 전체를 메모리에 담는다
- 대화 전체를 담기 때문에 대화가 길어지면 메모리 사용랴잉 많아지고 비용이 증가한다
- 자동완성 기능에 유용하다


In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

# 모든 대화를 저장한다.
memory.save_context({"input": "Hi!"},{"output":"How are you?"})
memory.save_context({"input": "Hi!"},{"output":"How are you?"})
memory.save_context({"input": "Hi!"},{"output":"How are you?"})
memory.load_memory_variables({})

### ConversationBufferWindowMemory

- 대화의 특정 부분만 저장하는 메모리 ex: 최근 5개의 대화 저장
- 메모리를 특정 사이즈로 유지하는데 장점
- 단점은 최근 대화만 기억할 수 있다는 점


In [ ]:
from langchain.memory import ConversationBufferWindowMemory

# k는 몇개의 대화를 저장할지에 대한 파라미터
memory = ConversationBufferWindowMemory(
  return_messages=True,
  k=4
)

def add_message(input, output):
  memory.save_context({"input":input},{"output":output})

add_message(1,1)
add_message(2,2)
add_message(3,3)
add_message(4,4)
add_message(5,5)

memory.load_memory_variables({})

### ConversationSummaryMemory

- llm을 사용함
- 메모리에 그냥 저장하는 것이 아니라 자체적으로 요약해서 저장함: 대화가 길어지면 토큰 비용을 절약할 수 있음


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory

chat = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryMemory(llm=chat)

def add_message(input, output):
  memory.save_context({"input":input},{"output":output})

def get_history():
  return memory.load_memory_variables({})

add_message("Hi I'm LMH, I live in South Korea","Wow that is so cool!")
add_message("South Korea is so pretty!","I wish I could go!!")
get_history()

### ConversationSummaryBufferMemory

- Summary Memory와 Buffer Memory의 결합
- 메모리가 저장되는 개수를 추적하고 토큰 한계에 다다르면 대화를 요약해서 저장함


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory

chat = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
  llm=chat, 
  max_token_limit=50, 
  return_messages=True
)

def add_message(input, output):
  memory.save_context({"input":input},{"output":output})

def get_history():
  return memory.load_memory_variables({})

add_message("Hi I'm LMH, I live in South Korea","Wow that is so cool!")
add_message("South Korea is so pretty!","I wish I could go!!")
add_message("South Korean food is so delicious","I wish I could eat them!!")
add_message("How far from Argentina?","I don't know! Super Far!")
get_history()

### ConversationKGMemory

- KG = Knowledge Graph
- 대화중의 엔티티의 KG를 생성함 : 가장 중요한 것만 뽑아내는 요약본 같은 것


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationKGMemory

chat = ChatOpenAI(temperature=0.1)
memory = ConversationKGMemory(
  llm=chat, 
  return_messages=True
)

def add_message(input, output):
  memory.save_context({"input":input},{"output":output})

add_message("Hi I'm LMH, I live in South Korea","Wow that is so cool!")
add_message("South Korea is so pretty!","I wish I could go!!")
add_message("South Korean food is so delicious","I wish I could eat them!!")
memory.load_memory_variables({"input" : "Tell me about South Korea"})


### LLM Chain

- off-the-shelf : 일반적인 목적을 가진 chain
- 이미 만들어진 chain으로 빠르게 시작할 때 좋지만 실제로는 커스텀해서 chain을 직접 만드는 경우가 더 많을 것


#### PromptTemplate 사용


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

chat = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
  llm=chat, 
  max_token_limit=100, 
  # return_messages=True,
  memory_key="chat_history"
)

template = """
  You are a helpful AI talking to a human

  {chat_history}
  Human:{question}
  You:
"""


chain = LLMChain(
  llm = chat,
  memory = memory,
  prompt=PromptTemplate.from_template(template),
  verbose=True
)

chain.predict(question="My name is Nico")
chain.predict(question="I live in Seoul")
chain.predict(question="What is my name?")

#### ChatPromptTemplate 사용


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

chat = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
  llm=chat, 
  max_token_limit=100, 
  return_messages=True,
  memory_key="chat_history"
)

prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful AI talking to a human"),
  MessagesPlaceholder(variable_name="chat_history"),
  ("human", "{question}")
])

chain = LLMChain(
  llm = chat,
  memory = memory,
  prompt=prompt,
  verbose=True
)

chain.predict(question="My name is Nico")
chain.predict(question="I live in Seoul")
chain.predict(question="What is my name?")

### LCEL Based Memory Chain


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

chat = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
  llm=chat, 
  max_token_limit=100, 
  return_messages=True,
)

prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful AI talking to a human"),
  MessagesPlaceholder(variable_name="history"),
  ("human", "{question}")
])

def load_memory(_):
  return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | chat

def invoke_chain(question):
  result = chain.invoke({
    "question":"My name is Nico"
  })
  memory.save_context({"input":question},{"output":result.content})
  print(result)

invoke_chain("My name is nico")
invoke_chain("What is my name?")

KeyError: 'chat_history'